In [1]:
# ekonlpy 설치
!pip install ekonlpy

# mecab-ko-dic 설치 (필요한 경우)
!apt-get install mecab-ko-dic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.2/33.2 MB 33.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.0/698.0 kB 33.4 MB/s eta 0:00:00
  Created wheel for mecab-ko-dic: filename=mecab_ko_dic-1.0.0-py3-none-any.whl size=33424393 sha256=ae6feeb8ec72ff40151f6ba599ebfc091c1afff8c01dcaea050b22924139b611
  Stored in directory: /root/.cache/pip/wheels/05/9d/25/2fd8c3b4ee5d7ed5b676121162e880a941864282d051cdf7a1
Successfully built mecab-ko-dic
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package mecab-ko-dic


In [8]:
import pandas as pd
from ekonlpy import Mecab # 경제 분야 특화 토크나이저

mecab = Mecab()

# ALLOWED_TAGS에 포함된 품사만 필터링
ALLOWED_TAGS = {'NNG', 'VA', 'VAX', 'MAG', 'negations'}

# CSV 파일 로드
df = pd.read_csv("/content/conferences_sentences.csv")

# 문자열 형태의 리스트를 실제 리스트로 변환
df["sentences"] = df["sentences"].apply(eval)  # 리스트 형태 변환

# 문장 단위로 데이터 풀기 (explode 사용)
df = df.explode("sentences").reset_index(drop=True)

# 날짜(date)를 기준으로 document_id 생성
df["document_id"] = df["date"].factorize()[0] + 1

# 각 document_id 내에서 문장 순서대로 sentence_id 추가
df["sentence_id"] = df.groupby("document_id").cumcount() + 1

# ekonply를 이용한 토큰화
def lemmatize_sentence(sentence):
    # 전체 형태소 분석
    all_tokens = [m[0] for m in mecab.pos(sentence)]

    # ALLOWED_TAGS에 해당하는 품사만 필터링
    filtered_tokens = [m[0] for m in mecab.pos(sentence) if m[1] in ALLOWED_TAGS]

    return " ".join(all_tokens), " ".join(filtered_tokens)

# 새로운 컬럼에 토큰화된 문장 및 필터링된 문장 추가
df[["tokenized_sentence", "filtered_tokenized_sentence"]] = df["sentences"].apply(lambda x: pd.Series(lemmatize_sentence(x)))

# 컬럼 정리 및 저장
df = df[["date", "document_id", "sentence_id", "tokenized_sentence", "filtered_tokenized_sentence"]]
print(df.head(20))

df.to_csv("press_conference_sentences.csv", index=False)

print("기자회견 문서 데이터 변환 및 토큰화 완료!")


          date  document_id  sentence_id  \
0   2009-04-09            1            1   
1   2009-04-09            1            2   
2   2009-04-09            1            3   
3   2009-04-09            1            4   
4   2009-04-09            1            5   
5   2009-04-09            1            6   
6   2009-05-12            2            1   
7   2009-05-12            2            2   
8   2009-05-12            2            3   
9   2009-05-12            2            4   
10  2009-05-12            2            5   
11  2009-05-12            2            6   
12  2009-06-11            3            1   
13  2009-06-11            3            2   
14  2009-06-11            3            3   
15  2009-06-11            3            4   
16  2009-06-11            3            5   
17  2009-06-11            3            6   
18  2009-07-09            4            1   
19  2009-07-09            4            2   

                                   tokenized_sentence  \
0   금융통화위원회 는 다음 통